<a href="https://colab.research.google.com/github/ikfaisal/dsia/blob/master/DSIA_Lab_2_1_3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.3 *Database Programming with Python and SQLite*

<a name="demo"></a>
# Using SQLite

## Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



## The `sqlite3` Command Line Utility

- useful for basic SQL tasks and databse maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands". 

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


## The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [6]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

In [2]:
pwd #path of file created

'C:\\Users\\YC\\Desktop\\IOD\\Mod2'

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [3]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()

OperationalError: table houses already exists

In [4]:
c.execute('DROP TABLE houses;')

# Save (commit) the changes:
conn.commit()

In [5]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()

In [ ]:
#asking sql to try creating the table, if it cannot then it will execute except
try: 
    c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
    conn.commit()
except:
    c.execute('DROP TABLE houses;')
    conn.commit()
    c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
    conn.commit()

With the database saved the table should now be viewable using SQLite Manager.

## Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [6]:
# A(1, 4000, 5, 22, 619000)
last_sale = (None, 4000, 5, 22, 619000)

# Execute Command
c.execute(f"insert into houses values(1, 4000, 5, 22, 619000);")
# Remember to commit the changes
conn.commit()

In [ ]:
last_sale = (None, 4000, 5, 22, 619000)

# Execute Command
c.execute(f'insert into houses values(?,?,?,?,?)',last_sale)
# Remember to commit the changes
conn.commit()

In [7]:
c.execute("SELECT * FROM houses")
print(c.fetchone())

(1, 4000, 5, 22, 619000)


In [8]:
last_sale = (2, 4000, 5, 22, 619000)

# Execute Command
c.execute(f"insert into houses values {last_sale};")
# Remember to commit the changes
conn.commit()

In [9]:
c.execute("SELECT * FROM houses")
c.fetchall()

[(1, 4000, 5, 22, 619000), (2, 4000, 5, 22, 619000)]

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key. 

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [10]:
# A #None is the Primary Key and we have alrdy set it to primary key and will auto-increment as explained on top
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
# 
c.executemany('insert into houses values(?,?,?,?,?)',recent_sales)
conn.commit()

Select all rows from houses

In [11]:
# A
c.execute("SELECT * FROM houses")
c.fetchall()

[(1, 4000, 5, 22, 619000),
 (2, 4000, 5, 22, 619000),
 (3, 2390, 4, 34, 319000),
 (4, 1870, 3, 14, 289000),
 (5, 1505, 3, 90, 269000)]

## Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more. 

In [1]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
data = genfromtxt('..\\DATA\\housing-data.csv', dtype = 'i8', 
                    delimiter = ',', skip_header = 1).tolist()

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [3]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [4]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [7]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [8]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

## Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [9]:
c.execute("SELECT * FROM houses")
c.fetchall()

[(1, 4000, 5, 22, 619000),
 (2, 4000, 5, 22, 619000),
 (3, 2390, 4, 34, 319000),
 (4, 1870, 3, 14, 289000),
 (5, 1505, 3, 90, 269000),
 (6, 2104, 3, 70, 399900),
 (7, 1600, 3, 28, 329900),
 (8, 2400, 3, 44, 369000),
 (9, 1416, 2, 49, 232000),
 (10, 3000, 4, 75, 539900),
 (11, 1985, 4, 61, 299900),
 (12, 1534, 3, 12, 314900),
 (13, 1427, 3, 57, 198999),
 (14, 1380, 3, 14, 212000),
 (15, 1494, 3, 15, 242500),
 (16, 1940, 4, 7, 239999),
 (17, 2000, 3, 27, 347000),
 (18, 1890, 3, 45, 329999),
 (19, 4478, 5, 49, 699900),
 (20, 1268, 3, 58, 259900),
 (21, 2300, 4, 77, 449900),
 (22, 1320, 2, 62, 299900),
 (23, 1236, 3, 78, 199900),
 (24, 2609, 4, 5, 499998),
 (25, 3031, 4, 21, 599000),
 (26, 1767, 3, 44, 252900),
 (27, 1888, 2, 79, 255000),
 (28, 1604, 3, 13, 242900),
 (29, 1962, 4, 53, 259900),
 (30, 3890, 3, 36, 573900),
 (31, 1100, 3, 60, 249900),
 (32, 1458, 3, 29, 464500),
 (33, 2526, 3, 13, 469000),
 (34, 2200, 3, 28, 475000),
 (35, 2637, 3, 25, 299900),
 (36, 1839, 2, 40, 349900),
 (3

In [18]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')
conn.commit()

QUIZ: What would `DELETE FROM houses` do?

In [ ]:
deleted field1 52,53 row

In [19]:
c.execute("SELECT * FROM houses")
c.fetchall()

[(1, 4000, 5, 22, 619000),
 (2, 4000, 5, 22, 619000),
 (3, 2390, 4, 34, 319000),
 (4, 1870, 3, 14, 289000),
 (5, 1505, 3, 90, 269000),
 (6, 2104, 3, 70, 399900),
 (7, 1600, 3, 28, 329900),
 (8, 2400, 3, 44, 369000),
 (9, 1416, 2, 49, 232000),
 (10, 3000, 4, 75, 539900),
 (11, 1985, 4, 61, 299900),
 (12, 1534, 3, 12, 314900),
 (13, 1427, 3, 57, 198999),
 (14, 1380, 3, 14, 212000),
 (15, 1494, 3, 15, 242500),
 (16, 1940, 4, 7, 239999),
 (17, 2000, 3, 27, 347000),
 (18, 1890, 3, 45, 329999),
 (19, 4478, 5, 49, 699900),
 (20, 1268, 3, 58, 259900),
 (21, 2300, 4, 77, 449900),
 (22, 1320, 2, 62, 299900),
 (23, 1236, 3, 78, 199900),
 (24, 2609, 4, 5, 499998),
 (25, 3031, 4, 21, 599000),
 (26, 1767, 3, 44, 252900),
 (27, 1888, 2, 79, 255000),
 (28, 1604, 3, 13, 242900),
 (29, 1962, 4, 53, 259900),
 (30, 3890, 3, 36, 573900),
 (31, 1100, 3, 60, 249900),
 (32, 1458, 3, 29, 464500),
 (33, 2526, 3, 13, 469000),
 (34, 2200, 3, 28, 475000),
 (35, 2637, 3, 25, 299900),
 (36, 1839, 2, 40, 349900),
 (3

## Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [10]:
# ANSWER
c.execute("SELECT * FROM houses WHERE bdrms = 4")
c.fetchall()

[(3, 2390, 4, 34, 319000),
 (10, 3000, 4, 75, 539900),
 (11, 1985, 4, 61, 299900),
 (16, 1940, 4, 7, 239999),
 (21, 2300, 4, 77, 449900),
 (24, 2609, 4, 5, 499998),
 (25, 3031, 4, 21, 599000),
 (29, 1962, 4, 53, 259900),
 (38, 2040, 4, 75, 314900),
 (40, 1811, 4, 24, 285900),
 (43, 2132, 4, 28, 345000),
 (44, 4215, 4, 66, 549000),
 (45, 2162, 4, 43, 287000),
 (48, 2567, 4, 57, 314000),
 (51, 1852, 4, 64, 299900),
 (56, 3000, 4, 75, 539900),
 (57, 1985, 4, 61, 299900),
 (62, 1940, 4, 7, 239999),
 (67, 2300, 4, 77, 449900),
 (70, 2609, 4, 5, 499998),
 (71, 3031, 4, 21, 599000),
 (75, 1962, 4, 53, 259900),
 (84, 2040, 4, 75, 314900),
 (86, 1811, 4, 24, 285900),
 (89, 2132, 4, 28, 345000),
 (90, 4215, 4, 66, 549000),
 (91, 2162, 4, 43, 287000),
 (94, 2567, 4, 57, 314000),
 (97, 1852, 4, 64, 299900)]

**2. Run a query to calculate the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [11]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1832.8846153846155, 324782.6153846154),
 (4, 2400.0689655172414, 375434.275862069),
 (5, 4239.0, 659450.0)]

# Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [12]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

## Writing data into a database

Data in Pandas can be loaded into a relational database. 

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [13]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('..\\DATA\\housing-data.csv', low_memory = False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


In [14]:
data.describe()

,sqft,bdrms,age,price
count,47.000000,47.000000,47.000000,47.000000
mean,2000.680851,3.170213,42.744681,340412.659574
std,794.702354,0.760982,22.873440,125039.899586
min,852.000000,1.000000,5.000000,169900.000000
25%,1432.000000,3.000000,24.500000,249900.000000
50%,1888.000000,3.000000,44.000000,299900.000000
75%,2269.000000,4.000000,61.500000,384450.000000
max,4478.000000,5.000000,79.000000,699900.000000


We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [15]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

Run a query to get the average price of each house size from this table:

In [16]:
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


In [17]:
c.execute("SELECT AVG(sqft), AVG(price)  FROM houses_pandas")
c.fetchall()

[(2000.6808510638298, 340412.6595744681)]

In [18]:
#ANSWER
c.execute("SELECT sqft, AVG(price)  FROM houses_pandas GROUP BY sqft  ")
c.fetchall()


[(852, 179900.0),
 (1000, 169900.0),
 (1100, 249900.0),
 (1200, 299000.0),
 (1203, 239500.0),
 (1236, 199900.0),
 (1239, 229900.0),
 (1268, 259900.0),
 (1320, 299900.0),
 (1380, 212000.0),
 (1416, 232000.0),
 (1427, 198999.0),
 (1437, 249900.0),
 (1458, 464500.0),
 (1494, 242500.0),
 (1534, 314900.0),
 (1600, 329900.0),
 (1604, 242900.0),
 (1664, 368500.0),
 (1767, 252900.0),
 (1811, 285900.0),
 (1839, 349900.0),
 (1852, 299900.0),
 (1888, 255000.0),
 (1890, 329999.0),
 (1940, 239999.0),
 (1962, 259900.0),
 (1985, 299900.0),
 (2000, 347000.0),
 (2040, 314900.0),
 (2104, 399900.0),
 (2132, 345000.0),
 (2162, 287000.0),
 (2200, 475000.0),
 (2238, 329900.0),
 (2300, 449900.0),
 (2400, 369000.0),
 (2526, 469000.0),
 (2567, 314000.0),
 (2609, 499998.0),
 (2637, 299900.0),
 (3000, 539900.0),
 (3031, 599000.0),
 (3137, 579900.0),
 (3890, 573900.0),
 (4215, 549000.0),
 (4478, 699900.0)]

In [19]:
c.execute("SELECT bdrms, AVG(price)  FROM houses_pandas GROUP BY bdrms")
c.fetchall()


[(1, 169900.0),
 (2, 280866.6666666667),
 (3, 326403.92),
 (4, 377449.78571428574),
 (5, 699900.0)]

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

2. When would you want to use Pandas on a dataset retrieved from a database?

# Reference: SQL Syntax 


## SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

## WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

## AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate funtions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

group by and over
https://www.analyticsvidhya.com/blog/2020/12/window-function-a-must-know-sql-concept/

# Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

## Read CSV into DataFrame

In [20]:
import pandas as pd
import sqlite3
from pandas.io import sql

In [21]:
# Reading CSV to Dataframe
orders = pd.read_csv('..\\DATA\\P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv('..\\DATA\\P12-OrderBreakdown.csv', encoding = 'utf-8')

In [22]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [23]:
orders_break_down.head()

,Order ID,Product Name,Discount,Sales,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


## Replace Space with Underscore in Column Names

In [24]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]

In [25]:
orders.head()

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [26]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

In [27]:
orders_break_down.head()

,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


## Check DataTypes 

In [61]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [62]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

## Save these two dataframes as a table in sqlite

In [29]:
# Establishing Local DB connection
db_connection = sqlite3.connect('..\\DATA\\eshop.db.sqlite')

# Save these two dataframes as a table in sqlite
orders.to_sql('orders_pandas', con=db_connection, if_exists = 'replace', index=False)
orders_break_down.to_sql('orders_break_down_pandas', con=db_connection, if_exists = 'replace', index=False)

## Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [ ]:
# Select first 10 rows of all columns in orders table
# this will fail coz the data name is orders_pandas

query = """
        SELECT * 
        FROM orders
        LIMIT 10;
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

In [72]:
# Select first 10 rows of all columns in orders table


query = """
        SELECT * 
        FROM orders_pandas 
        LIMIT 10;
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


In [77]:
# Select Number of Orders for Each Customer
query = """
        SELECT COUNT(order_id) as Num_of_orders, customer_name
        FROM orders_pandas 
        GROUP BY customer_name;
        """
NumOfOrders = sql.read_sql(query, con=db_connection)
NumOfOrders

,Num_of_orders,customer_name
0,11,Aaron Bergman
1,7,Aaron Hawkins
2,12,Aaron Smayling
3,4,Adam Bellavance
4,8,Adam Hart
...,...,...
787,2,Xylona Preis
788,5,Yana Sorensen
789,11,Yoseph Carroll
790,9,Zuschuss Carroll


## Select Number of Customers for Each Country

In [30]:
# Select Number of Customers for Each Country
#count(distinct(customer_name)) to use distinct customer_name
query = """
        SELECT COUNT(country) as Num_of_customer, country
        FROM orders_pandas 
        GROUP BY country;
        """
NumOfCountry = sql.read_sql(query, con=db_connection)
NumOfCountry

,Num_of_customer,country
0,135,Austria
1,68,Belgium
2,29,Denmark
3,34,Finland
4,991,France
5,806,Germany
6,50,Ireland
7,493,Italy
8,194,Netherlands
9,37,Norway


## Select discount, sales, quantity for Each Order from orders_break_down Table

In [78]:
orders_break_down.head()

,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


In [83]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table
query = """
        SELECT order_id,discount, sales, quantity
        FROM orders_break_down_pandas
        GROUP BY order_id;
        """

queryinfo = sql.read_sql(query, con=db_connection)
queryinfo

,order_id,discount,sales,quantity
0,ES-2011-1001989,0.1,229.0,9
1,ES-2011-1010958,0.0,58.0,2
2,ES-2011-1012469,0.0,148.0,3
3,ES-2011-1043483,0.0,224.0,11
4,ES-2011-1058269,0.0,13.0,2
...,...,...,...,...
4112,IT-2014-5966070,0.5,107.0,4
4113,IT-2014-5975833,0.1,656.0,2
4114,IT-2014-5984498,0.5,83.0,1
4115,IT-2014-5989338,0.1,1603.0,9


##  Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [35]:
# Select discount, sales, quantity, total price for Each Order from orders_break_down Table
query = """
        SELECT order_id,discount, sales, quantity,
        (sales*quantity-discount) as Total_Price,
        (sales*quantity*(1-discount)) as percentage_Total_Price
        FROM orders_break_down_pandas
        GROUP BY order_id
        ORDER by total_price DESC;
        """

queryinfo2 = sql.read_sql(query, con=db_connection)
queryinfo2

,order_id,discount,sales,quantity,Total_Price,percentage_Total_Price
0,ES-2014-3785216,0.1,5729.0,14,80205.9,72185.4
1,ES-2011-3248922,0.0,6517.0,12,78204.0,78204.0
2,ES-2013-2860574,0.0,5274.0,10,52740.0,52740.0
3,ES-2012-5671193,0.0,5725.0,9,51525.0,51525.0
4,IT-2011-1978668,0.1,4544.0,11,49983.9,44985.6
...,...,...,...,...,...,...
4112,IT-2014-2273520,0.5,6.0,1,5.5,3.0
4113,ES-2014-2457136,0.0,5.0,1,5.0,5.0
4114,ES-2013-4933394,0.0,5.0,1,5.0,5.0
4115,IT-2014-3944375,0.5,4.0,1,3.5,2.0


## Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [86]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100
query = """
        SELECT order_id,discount, sales, quantity,(sales*quantity-discount) as Total_Price
        FROM orders_break_down_pandas
        WHERE Total_Price > 100
        GROUP BY order_id
        ;
        """

queryinfo3 = sql.read_sql(query, con=db_connection)
queryinfo3

,order_id,discount,sales,quantity,Total_Price
0,ES-2011-1001989,0.1,229.0,9,2060.9
1,ES-2011-1010958,0.0,58.0,2,116.0
2,ES-2011-1012469,0.0,148.0,3,444.0
3,ES-2011-1043483,0.0,224.0,11,2464.0
4,ES-2011-1058269,0.1,748.0,4,2991.9
...,...,...,...,...,...
3654,IT-2014-5965314,0.5,60.0,4,239.5
3655,IT-2014-5966070,0.5,107.0,4,427.5
3656,IT-2014-5975833,0.1,656.0,2,1311.9
3657,IT-2014-5989338,0.1,1603.0,9,14426.9


## Select All Customer And The Product They Have Bought

In [87]:
orders_break_down.head()

,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


In [107]:
orders.head()

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [37]:
# Select All Customer And The Product They Have Bought
query = """
        SELECT o.customer_name, ob.product_name
        FROM orders_break_down_pandas ob
        LEFT JOIN orders_pandas o
        ON o.order_id = ob.order_id
        GROUP BY o.customer_name,ob.product_name;
        """

queryinfo4 = sql.read_sql(query, con=db_connection)
queryinfo4

,customer_name,product_name
0,Aaron Bergman,"Apple Office Telephone, Cordless"
1,Aaron Bergman,"Belkin Keyboard, Ergonomic"
2,Aaron Bergman,"Binney & Smith Markers, Fluorescent"
3,Aaron Bergman,"Brother Fax and Copier, Color"
4,Aaron Bergman,"Cisco Office Telephone, Cordless"
...,...,...
8005,Zuschuss Donatelli,"Memorex Numeric Keypad, Bluetooth"
8006,Zuschuss Donatelli,"Office Star Executive Leather Armchair, Adjust..."
8007,Zuschuss Donatelli,"Okidata Calculator, Red"
8008,Zuschuss Donatelli,"SanDisk Parchment Paper, Premium"


In [38]:
# Select All Customer And The Product They Have Bought
query = """
        SELECT customer_name, product_name
        FROM orders_break_down_pandas
        JOIN orders_pandas
            ON orders_break_down_pandas.order_id = orders_pandas.order_id
        GROUP BY customer_name,product_name;
        """

queryinfo4 = sql.read_sql(query, con=db_connection)
queryinfo4

,customer_name,product_name
0,Aaron Bergman,"Apple Office Telephone, Cordless"
1,Aaron Bergman,"Belkin Keyboard, Ergonomic"
2,Aaron Bergman,"Binney & Smith Markers, Fluorescent"
3,Aaron Bergman,"Brother Fax and Copier, Color"
4,Aaron Bergman,"Cisco Office Telephone, Cordless"
...,...,...
8005,Zuschuss Donatelli,"Memorex Numeric Keypad, Bluetooth"
8006,Zuschuss Donatelli,"Office Star Executive Leather Armchair, Adjust..."
8007,Zuschuss Donatelli,"Okidata Calculator, Red"
8008,Zuschuss Donatelli,"SanDisk Parchment Paper, Premium"


In [45]:
# Select All Customer And The Product They Have Bought
query = """
        SELECT o.customer_name, 
        GROUP_CONCAT(ob.product_name) as products
        FROM orders_break_down_pandas ob
        INNER JOIN orders_pandas o
        ON ob.order_id = o.order_id
        GROUP BY o.customer_name;
        """

queryinfo4 = sql.read_sql(query, con=db_connection)
queryinfo4

,customer_name,products
0,Aaron Bergman,"Harbour Creations Round Labels, Laser Printer ..."
1,Aaron Hawkins,"Xerox Note Cards, Recycled,Epson Inkjet, Wirel..."
2,Aaron Smayling,"Binney & Smith Pens, Fluorescent,Apple Signal ..."
3,Adam Bellavance,"Sauder Classic Bookcase, Mobile,Panasonic Prin..."
4,Adam Hart,"Cardinal Binder Covers, Economy,Ibico Binder C..."
...,...,...
787,Xylona Preis,"Ikea Classic Bookcase, Mobile,Smead Shelving, ..."
788,Yana Sorensen,"Rubbermaid Photo Frame, Durable,Binney & Smith..."
789,Yoseph Carroll,"Hon Round Labels, 5000 Label Set,Smead Folders..."
790,Zuschuss Carroll,"Breville Microwave, Silver,Cuisinart Stove, Si..."


## Select Number of 'Furniture' Orders For Each Country 

In [94]:
orders.head()

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [115]:
# Select Number of 'Furniture' Orders For Each Country

query = """
        SELECT COUNT(category) as num_of_furniture_order, country
        FROM orders_break_down_pandas
        JOIN orders_pandas
            ON orders_break_down_pandas.order_id = orders_pandas.order_id
        WHERE category = 'Furniture'
        GROUP BY country;
        """

queryinfo5 = sql.read_sql(query, con=db_connection)
queryinfo5

,num_of_furniture_order,country
0,40,Austria
1,20,Belgium
2,9,Denmark
3,16,Finland
4,299,France
5,264,Germany
6,12,Ireland
7,136,Italy
8,76,Netherlands
9,6,Norway


In [52]:
# Select Number of 'Furniture' Orders For Each Country

query = """
        SELECT COUNT(quantity) as num_furniture, 
        SUM(quantity) as sum_furniture,
        COUNT(DISTINCT((orders_break_down_pandas.order_id))) as distinct_furniture,
        country
        FROM orders_break_down_pandas
        INNER JOIN orders_pandas
        ON orders_break_down_pandas.order_id = orders_pandas.order_id
        WHERE category = 'Furniture'
        GROUP BY country;
        """

queryinfo5 = sql.read_sql(query, con=db_connection)
queryinfo5

,num_furniture,sum_furniture,distinct_furniture,country
0,40,132,33,Austria
1,20,66,19,Belgium
2,9,30,8,Denmark
3,16,53,14,Finland
4,299,1145,260,France
5,264,966,224,Germany
6,12,49,10,Ireland
7,136,502,117,Italy
8,76,310,63,Netherlands
9,6,36,6,Norway


## Select Number of 'Furniture' Orders For The Country Denmark

In [53]:
# Select Number of 'Furniture' Orders For The Country Denmark
query = """
        SELECT COUNT(category) as num_of_furniture_orderS, country
        FROM orders_break_down_pandas
        JOIN orders_pandas
            ON orders_break_down_pandas.order_id = orders_pandas.order_id
        WHERE category = 'Furniture'
        AND country = 'Denmark';
        """

queryinfo6 = sql.read_sql(query, con=db_connection)
queryinfo6

,num_of_furniture_orderS,country
0,9,Denmark


## Select Total Sales With Discount and Without Discount for Each Country

In [117]:
orders.head(2)

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom


In [118]:
orders_break_down.head(2)

,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture


In [56]:
# Select Total Sales With Discount and Without Discount for Each Country
query = """
        SELECT country
        ,SUM(CASE WHEN discount == 0 THEN (sales*quantity) ELSE 0 END) as non_discount_sales
        ,SUM(CASE WHEN discount > 0 THEN (sales*quantity*(1-discount)) ELSE 0 END) as discount_sales
        FROM orders_break_down_pandas
        JOIN orders_pandas
            ON orders_break_down_pandas.order_id = orders_pandas.order_id
        GROUP BY country
        ;
        """

queryinfo7 = sql.read_sql(query, con=db_connection)
queryinfo7

,country,non_discount_sales,discount_sales
0,Austria,391694.0,0.0
1,Belgium,229829.0,0.0
2,Denmark,0.0,17117.8
3,Finland,82001.0,0.0
4,France,665914.0,2063927.0
5,Germany,1206817.0,1026641.8
6,Ireland,0.0,37783.6
7,Italy,806777.0,214253.0
8,Netherlands,0.0,195750.9
9,Norway,99878.0,0.0


In [61]:

query = """
SELECT orders_pandas.country, SUM(orders_break_down_pandas.sales) AS total_quantity, 
SUM(orders_break_down_pandas.sales) FILTER(WHERE orders_break_down_pandas.discount==0) AS "no_disc", 
SUM(orders_break_down_pandas.sales) FILTER(WHERE orders_break_down_pandas.discount>0) AS "disc" 
FROM orders_pandas 
INNER JOIN orders_break_down_pandas ON orders_pandas.order_id = orders_break_down_pandas.order_id 
GROUP BY orders_pandas.country;
"""
queryinfo7 = sql.read_sql(query, con=db_connection)
queryinfo7

,country,total_quantity,no_disc,disc
0,Austria,79382.0,79382.0,NaN
1,Belgium,42320.0,42320.0,NaN
2,Denmark,7763.0,NaN,7763.0
3,Finland,20702.0,20702.0,NaN
4,France,609683.0,129791.0,479892.0
5,Germany,488681.0,245424.0,243257.0
6,Ireland,15998.0,NaN,15998.0
7,Italy,252742.0,174097.0,78645.0
8,Netherlands,70313.0,NaN,70313.0
9,Norway,20529.0,20529.0,NaN


## Select Total Quantity, Total Sales for Each Country

In [190]:
# Select Total Quantity, Total Sales for Each Country
query = """
        SELECT SUM(quantity) as Total_quantity,sum(sales) as Total_sales, country
        
        FROM orders_break_down_pandas
        JOIN orders_pandas
            ON orders_break_down_pandas.order_id = orders_pandas.order_id
        GROUP BY country
        ;
        """

queryinfo8 = sql.read_sql(query, con=db_connection)
queryinfo8

,Total_quantity,Total_sales,country
0,973,79382.0,Austria
1,532,42320.0,Belgium
2,204,7763.0,Denmark
3,201,20702.0,Finland
4,7329,609683.0,France
5,6179,488681.0,Germany
6,392,15998.0,Ireland
7,3612,252742.0,Italy
8,1526,70313.0,Netherlands
9,261,20529.0,Norway


## Select Top 3 Country Based on Sales

In [63]:
# Select Top 3 Country Based on Sales

query = """
        SELECT sum(sales*quantity) as Total_sales, country
        
        FROM orders_break_down_pandas
        JOIN orders_pandas
            ON orders_break_down_pandas.order_id = orders_pandas.order_id
        GROUP BY country
        ORDER BY Total_sales DESC
        LIMIT 3;
        """

queryinfo9 = sql.read_sql(query, con=db_connection)
queryinfo9

,Total_sales,country
0,3108592.0,France
1,2411483.0,Germany
2,2161947.0,United Kingdom


## Select Bottom 3 Country Based On Quantities

In [62]:
# Select Bottom 3 Country Based On Quantities
query = """
        SELECT SUM(quantity) as Total_quantity, country
        
        FROM orders_break_down_pandas
        JOIN orders_pandas
            ON orders_break_down_pandas.order_id = orders_pandas.order_id
        GROUP BY country
        ORDER BY Total_quantity ASC
        LIMIT 3;
        """

queryinfo10 = sql.read_sql(query, con=db_connection)
queryinfo10

,Total_quantity,country
0,827,Finland
1,880,Denmark
2,1283,Norway


## Select Average Sales By Categroy For The Country 'France'

In [192]:
orders_break_down.head()

,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


In [64]:
# Select Average Sales By Categroy For The Country 'France'
query = """
        SELECT AVG(sales*quantity) as AVG_sales, category, country
        
        FROM orders_break_down_pandas
        JOIN orders_pandas
            ON orders_break_down_pandas.order_id = orders_pandas.order_id
        WHERE country = 'France'
        GROUP BY category
        ;
        """

queryinfo11 = sql.read_sql(query, con=db_connection)
queryinfo11

,AVG_sales,category,country
0,3091.789298,Furniture,France
1,847.214460,Office Supplies,France
2,2956.544041,Technology,France


## Select Country, Category and Total Sales Where Average Total Sales is The Highest

In [187]:
orders_break_down.head(2)

,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture


In [70]:
# Select Country, Category and Total Sales Where Average Total Sales is The Highest
query = """
        SELECT country,  category, 
        SUM(sales*quantity) as Total_sales , 
        AVG(sales*quantity) as Average_sales
        
        FROM orders_break_down_pandas
        JOIN orders_pandas
            ON orders_break_down_pandas.order_id = orders_pandas.order_id
        GROUP BY category,country
        ORDER BY Average_sales DESC
        LIMIT 1;
        """

queryinfo12 = sql.read_sql(query, con=db_connection)
queryinfo12

,country,category,Total_sales,Average_sales
0,Switzerland,Technology,84342.0,6487.846154


In [67]:
# Select Country, Category and Total Sales Where Average Total Sales is The Highest
query = """
        SELECT country,  category, SUM(sales) as Total_sales , AVG(sales) as Average_sales,
        RANK() OVER(PARTITION BY country ORDER BY AVG(sales) DESC) Rank
        FROM orders_break_down_pandas
        JOIN orders_pandas
            ON orders_break_down_pandas.order_id = orders_pandas.order_id
        GROUP BY category,country
        ORDER BY country, Average_sales DESC 
        ;
        """

queryinfo12 = sql.read_sql(query, con=db_connection)
queryinfo12

,country,category,Total_sales,Average_sales,Rank
0,Austria,Technology,27086.0,576.297872,1
1,Austria,Furniture,21091.0,527.275000,2
2,Austria,Office Supplies,31205.0,176.299435,3
3,Belgium,Technology,13591.0,715.315789,1
4,Belgium,Furniture,8870.0,443.500000,2
5,Belgium,Office Supplies,19859.0,206.864583,3
6,Denmark,Technology,2608.0,217.333333,1
7,Denmark,Furniture,1471.0,163.444444,2
8,Denmark,Office Supplies,3684.0,94.461538,3
9,Finland,Furniture,9602.0,600.125000,1


In [72]:
# Select Country, Category and Total Sales Where Average Total Sales is The Highest
query = """
        SELECT country,  category, SUM(sales) as Total_sales , AVG(sales) as Average_sales
        
        FROM orders_break_down_pandas
        JOIN orders_pandas
            ON orders_break_down_pandas.order_id = orders_pandas.order_id
        GROUP BY category,country
        ORDER BY Average_sales DESC
        LIMIT 1;
        """
    
queryinfo12 = sql.read_sql(query, con=db_connection)
queryinfo12

,country,category,Total_sales,Average_sales
0,Switzerland,Technology,11737.0,902.846154


In [ ]:
# Select Country, Category and Total Sales Where Average Total Sales is The Highest
query = """
        SELECT country,  category, 
        SUM(sales) as Total_sales , AVG(sales) as Average_sales,
        RANK() OVER(PARTITION BY country ORDER BY AVG(sales) DESC) Rank
        FROM orders_break_down_pandas
        JOIN orders_pandas
            ON orders_break_down_pandas.order_id = orders_pandas.order_id
        GROUP BY category,country
        ORDER BY country, Average_sales DESC 
        ;
        """

queryinfo12 = sql.read_sql(query, con=db_connection)
queryinfo12

In [197]:
orders.columns

Index(['order_id', 'order_date', 'customer_name', 'country'], dtype='object')

In [198]:
orders_break_down.columns

Index(['order_id', 'product_name', 'discount', 'sales', 'quantity',
       'category'],
      dtype='object')

# JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams. 
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

#  ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL zoo](http://www.sqlzoo.net)  Great for learning syntax